In [1]:
import sys


no_of_cpu =8
max_cores =16
executor_mem = '56g'


Job_Name = 'Nishant: Summary_of_Summary'


from pyspark.mllib.stat import Statistics
from pyspark.sql.functions import asc,lit
#warnings.filterwarnings('error')
import pyspark
from datetime import datetime,timedelta
from pyspark.sql import SQLContext
from pyspark import SparkContext, SparkConf
from pyspark.sql import functions as F

conf = pyspark.SparkConf()
#import numpy
import calendar
#import pandas as pd
#import simplejson as json
#import pandas as pd
import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType
from pyspark.sql.functions import lit
import simplejson as json
import json, pprint, requests
#es_nodes = '10.35.12.5'
#es_nodes = '10.35.12.6'
es_nodes = '10.35.12.194'#,10.35.12.6,10.35.12.5
es_nodes_temp='10.35.12.194'
es_port = '5432'
es_user = 'elastic'
es_pwd = 'bEiilauM3es'
mesos_ip = 'mesos://10.33.195.18:5050'#'mesos://10.35.12.5:5050'

spark.stop() #############NEED TO COMMENT THIS SPARK.STOP WHEN RUNNING THROUGH SHELL###############################

conf.setMaster(mesos_ip)

conf.set('spark.executor.cores',no_of_cpu) ### change 1
#conf.set('spark.memory.fraction','.2')
conf.set('spark.executor.memory',executor_mem) 
conf.set('spark.es.scroll.size','10000')
conf.set('spark.network.timeout','600s')
conf.set('spark.sql.crossJoin.enabled', 'true')

conf.set('spark.ui.port','4052')

conf.set('spark.executor.heartbeatInterval','60s')
conf.set("spark.driver.cores","4")
conf.set("spark.driver.extraJavaOptions","-Xmx4g -Xms4g")

#conf.set("spark.shuffle.blockTransferService", "nio"); 
conf.set("spark.files.overwrite","true");
conf.set("spark.kryoserializer.buffer", "70"); 
conf.set("spark.driver.extraJavaOptions", "-XX:+UseG1GC");
conf.set("spark.executor.extraJavaOptions", "-XX:+UseG1GC");
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer"); 
conf.set("spark.broadcast.compress", "true"); 
conf.set("spark.shuffle.compress", "true"); 
conf.set("spark.shuffle.spill.compress", "true");
conf.set("spark.app.name", Job_Name)
#conf.set("spark.io.compression.codec","org.apache.spark.io.LZ4CompressionCodec");
#conf.set("spark.sql.inMemoryColumnarStorage.compressed", "true"); 
from pyspark.sql.functions import broadcast

conf.set('spark.driver.memory','20g') ### change 2
conf.set('spark.cores.max',max_cores) ### change 3
conf.set('spark.sql.shuffle.partitions','400')
#conf.set('spark.sql.crossJoin.enabled', 'true')
conf.set('es.nodes',es_nodes)
conf.set('es.port',es_port)
conf.set('es.nodes.wan.only','true')
conf.set("spark.sql.autoBroadcastJoinThreshold",-1)

#conf.set('spark.es.net.http.auth.user','Spark')
#conf.set('spark.es.net.http.auth.pass','Jarkpet1Sap3')
conf.set('spark.num.executors','4')
conf.set('spark.debug.maxToStringFields', 200)

conf.set('spark.es.net.http.auth.user', es_user)
conf.set('spark.es.net.http.auth.pass', es_pwd)

conf.set('spark.es.mapping.date.rich','false')
spark = pyspark.SparkContext(conf=conf)
sqlContext = SQLContext(spark)

# Load Data into PySpark DataFrames
# Prodcom Data Frame
import json, pprint, requests
import pyspark.sql.functions as sf
from pyspark.sql.types import *


import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType



Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
19253,None,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from pyspark.sql.functions import hash,col,regexp_replace,collect_list,concat_ws,initcap,count,least
import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType
import psycopg2
import time
from time import gmtime, strftime
import pytz
starttime = time.time()
start_time = datetime.datetime.now(pytz.timezone('Asia/Kolkata')).strftime("%Y-%m-%d %H:%M:%S.%f")
start_time

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'2024-03-23 22:32:32.016733'

In [3]:
prod_url = "jdbc:postgresql://10.35.12.194:5432/gpadmin"
prod_host = '10.35.12.194'
prod_port = '5432'
prod_dbname = 'gpadmin'
user_prod="gpcustcoe"
pwd_prod="gpcustcoe@123"
dbschema="public"


prod_gpdb_spark_options ={
    "url": "jdbc:postgresql://{host}:{port}/{dbname}".format(host=prod_host,port=prod_port, dbname=prod_dbname),
    "user": "{user}".format(user=user_prod),
    "password": "{password}".format(password=pwd_prod)
} 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
def load_gpdb_jdbc(col_str,dbtable,dbschema,col_name=None,time_filter=None,partitionColumn="row_num"):
    """
    This is used to read gpdb with filter for columns and can apply other filter(date,values).
    Time filter contains startdate,enddate
    """
    gscPythonOptions = {
                        "url": prod_url,
                        "user": user_prod,
                        "password": pwd_prod,
                        "dbschema": dbschema,
                        "dbtable": dbtable,
                        "partitionColumn":partitionColumn,
                        "partitions": 8,
                        "server.port":"1150-1170"}
   
    if time_filter:
        data = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                .selectExpr(col_str).drop_duplicates().filter(col(col_name).between(to_timestamp(lit(time_filter['start_date']),
                                                                 format='yyyy-MM-dd'),
                                                    to_timestamp(lit(time_filter['end_date']),
                                                                 format='yyyy-MM-dd')))
    else :
        data = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                .selectExpr(col_str).drop_duplicates()
   
    return data

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [61]:
def get_latest_progress():
    
    try: 
        gscPythonOptions = {
                 "url":prod_url ,
                 "user":user_prod ,
                 "password": pwd_prod,
                 "dbschema":"customermart",
                 "dbtable": "progress",#table change
                 "server.port":"1150-1170"} 

        list_tn=['gen_ai_summary']
        
        # this query will fetch till what date we have inserted the records in target
        last_run= sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
            .select('table_name','source','to_datetime').filter(col('table_name').isin(list_tn)).filter(col('status')=='Success')\
        .filter(col('records')!=0)
#         .filter(col('table_name')=='header')\
#             .filter(col('source')=='header')
        
        max_df=last_run.groupBy("table_name").agg(max("to_datetime").alias("datetime"))
        
        Max_last_run =max_df.select(min('datetime')).first()[0]


        # if no record is available in progress then it will raise value error, in this case default value will be picked from except clause
        if Max_last_run is None:
            raise ValueError("No records found!")

        #print("Record Found. Progress updated till {}".format(Max_last_run))

    except Exception as E: 
        Max_last_run = datetime.datetime(1900, 3, 12, 0, 0)  # this is the default start date when no record is present in progress for customer_demographics table
        print("Executed_except",repr(E))
    return Max_last_run

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [53]:
# Generic function to save the progress from last run into Progress table
def update_progress(table_name,source,time_filter,max_commit_timestamp,records,start_time,starttime,status):
    output_index = "progress"
    schema = "customermart"


    import sys
    import time
    try:
        


        df_progress= sqlContext.createDataFrame([(table_name,source,
                                       time_filter['start_date'].strftime("%Y-%m-%d %H:%M:%S"),
                                        max_commit_timestamp.strftime("%Y-%m-%d %H:%M:%S"),records)]
                                     ,['table_name', 'source','from_datetime','to_datetime','records'])\
        .withColumn('start_time',lit(start_time)).withColumn('end_time',current_timestamp())\
                    .withColumn('executed_in_mins',lit((time.time() - starttime)/60))\
                                .withColumn('status',lit(status)).withColumn('remarks',lit(None))
    

        
        df_progress.write.format("greenplum")\
            .option("dbtable",output_index).option('dbschema',schema)\
            .option("server.port","1150-1170").option("url",prod_url)\
            .option("user", user_prod).option("password",pwd_prod).mode('append').save()
        
       
    except Exception as e:
        x = e
        print(x)
    else:
        x = 'success'
        print("Updated Progress for {table_name} from {source} until {to_datetime}. {records} records processed in this run".format(table_name=table_name, source=source,to_datetime=max_commit_timestamp.strftime("%Y-%m-%d %H:%M:%S"),records=records))
        print(x)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [62]:

latest_progress=get_latest_progress()
# dates in string format
str_d1 = latest_progress.strftime("%Y-%m-%d")
str_d2 = datetime.datetime.now(pytz.timezone('Asia/Kolkata')).strftime("%Y-%m-%d")

# convert string to date object
d1 = datetime.datetime.strptime(str_d1, "%Y-%m-%d")
d2 = datetime.datetime.strptime(str_d2, "%Y-%m-%d")

# difference between dates in timedelta
delta = d2 - d1
day=delta.days-1 #currently T-2 change 2 to 1 for T-1

# Executed_except ValueError('No records found!')
# {'start_date': datetime.datetime(2018, 10, 30, 0, 0), 'end_date': datetime.datetime(2023, 9, 25, 0, 0)}

time_filter={'start_date':latest_progress,'end_date':latest_progress+datetime.timedelta(days=day)}
time_filter

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'start_date': datetime.datetime(2024, 3, 22, 0, 0), 'end_date': datetime.datetime(2024, 3, 22, 0, 0)}

In [29]:
col_str=['requestor_primary_number__c as mobile','customer_code__c as customer_code','reference_number_proposalnumber_quote_nu__c as reference_num1','spa_reference_number__c as reference_num2',
         'classification_disposition__c','sub_classification_disposition__c as sub_deposition','status as ticket_status','case_stage__c as ticket_stage','assigned_team__c as assigned_team',\
         'resolution_remarks__c as resolution_remarks','status_resolve_date__c as resolution_date',
           'ticket_source__c as ticket_source','complaint_registration_date__c as interaction_date','closeddate as ticket_close_date','status_closed_date__c','requestor_email_id__c as email',\
           'policy_number__c as policy_no','problem_remarks__c as remarks','ticket_status__c as open_status','sf_ticket_number__c as ticket_no','requester_name__c as customer_name',\
           'ageing_count_in_days__c as ticket_tat','certificate_number__c as certificate_no','id','load_date']
sc=load_gpdb_jdbc(col_str,"salesforce_case",'public','load_date',time_filter).filter((col('policy_no').isNotNull())|(col('customer_code').isNotNull()))
sc.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['mobile', 'customer_code', 'reference_num1', 'reference_num2', 'classification_disposition__c', 'sub_deposition', 'ticket_status', 'ticket_stage', 'assigned_team', 'resolution_remarks', 'resolution_date', 'ticket_source', 'interaction_date', 'ticket_close_date', 'status_closed_date__c', 'email', 'policy_no', 'remarks', 'open_status', 'ticket_no', 'customer_name', 'ticket_tat', 'certificate_no', 'id', 'load_date']

In [30]:
gscPythonOptions = {
         "url": "jdbc:postgresql://10.35.12.194:5432/gpadmin","user": "gpspark","password": "spark@456",
         "dbschema": "public","dbtable": "salesforce_emailmessage",
         "server.port":"1177"} 
sc_email =sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
.select('textbody','subject','parentid','messagedate')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
sc=sc.join(sc_email,sc_email.parentid==sc.id,'left').drop(sc_email.parentid).drop(sc.id)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
sc=sc.withColumn('certificate_no',when((col('certificate_no').isin('0','0.0','/00000','000000','00000000'))|(col('certificate_no').isNull()),lit('00000000')).otherwise(lit(col('certificate_no'))))\
.withColumn('policy_no',lpad(col('policy_no'),10,'0'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
gscPythonOptions = {
         "url": "jdbc:postgresql://10.35.12.194:5432/gpadmin","user": "gpspark","password": "spark@456",
         "dbschema": "registers","dbtable": "premium_register",
         "partitionColumn":"row_num","partitions":16,
         "server.port":"1150-1170"} 
df_pr =sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
.select('policy_no','customer_id','certificate_no')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
df_pr=df_pr.withColumn('certificate_no',when((col('certificate_no').isin('0','0.0','/00000','000000','00000000'))|(col('certificate_no').isNull()),lit('00000000')).otherwise(lit(col('certificate_no'))))\
.withColumn('policy_no',lpad(col('policy_no'),10,'0'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
sc_1 = sc.filter((sc.customer_code.like('T-%'))|(col('customer_code').isNull()))
sc_2 = sc.filter(~sc.customer_code.like('T-%'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
sc_1=sc_1.join(df_pr,(df_pr.policy_no==sc_1.policy_no)&(trim(df_pr.certificate_no)==trim(sc_1.certificate_no)),'inner').drop(df_pr.policy_no).drop(df_pr.certificate_no)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
sc_1.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['mobile', 'customer_code', 'reference_num1', 'reference_num2', 'classification_disposition__c', 'sub_deposition', 'ticket_status', 'ticket_stage', 'assigned_team', 'resolution_remarks', 'resolution_date', 'ticket_source', 'interaction_date', 'ticket_close_date', 'status_closed_date__c', 'email', 'policy_no', 'remarks', 'open_status', 'ticket_no', 'customer_name', 'ticket_tat', 'certificate_no', 'load_date', 'textbody', 'subject', 'messagedate', 'customer_id']

In [38]:
sc_1=sc_1.select('mobile', 'customer_code', 'reference_num1', 'reference_num2', 'classification_disposition__c', 'sub_deposition', 'ticket_status', 'ticket_stage', 'assigned_team', 'resolution_remarks', 'resolution_date', 'ticket_source', 'interaction_date', 'ticket_close_date', 'status_closed_date__c', 'email', 'policy_no', 'remarks', 'open_status', 'ticket_no', 'customer_name', 'ticket_tat', 'certificate_no', 'textbody', 'subject', 'messagedate','load_date')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
sc_2=sc_2.select('mobile', 'customer_code', 'reference_num1', 'reference_num2', 'classification_disposition__c', 'sub_deposition', 'ticket_status', 'ticket_stage', 'assigned_team', 'resolution_remarks', 'resolution_date', 'ticket_source', 'interaction_date', 'ticket_close_date', 'status_closed_date__c', 'email', 'policy_no', 'remarks', 'open_status', 'ticket_no', 'customer_name', 'ticket_tat', 'certificate_no', 'textbody', 'subject', 'messagedate','load_date')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
df_main=sc_1.union(sc_2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
gscPythonOptions = {"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
                    "user": "gpcustcoe",
                    "password": "gpcustcoe@123",
                    "dbschema": "customermart",
                    "dbtable": "header",
                    "server.port": '1150-1170'}

mbdp = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                   .selectExpr("source_system_customer_id as customer_code", "golden_id","active_flag").filter(col('active_flag')==1).drop('active_flag')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
df_main=df_main.join(mbdp,'customer_code','inner')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
df_main.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['customer_code', 'mobile', 'reference_num1', 'reference_num2', 'classification_disposition__c', 'sub_deposition', 'ticket_status', 'ticket_stage', 'assigned_team', 'resolution_remarks', 'resolution_date', 'ticket_source', 'interaction_date', 'ticket_close_date', 'status_closed_date__c', 'email', 'policy_no', 'remarks', 'open_status', 'ticket_no', 'customer_name', 'ticket_tat', 'certificate_no', 'textbody', 'subject', 'messagedate', 'load_date', 'golden_id']

In [43]:
df_temp=df_main.select('golden_id','customer_code', 'mobile', 'reference_num1', 'reference_num2', 'classification_disposition__c', 'sub_deposition', 'ticket_status', 'ticket_stage', 'assigned_team', 'resolution_remarks', 'resolution_date', 'ticket_source', 'interaction_date', 'ticket_close_date', 'status_closed_date__c', 'email', 'policy_no', 'remarks', 'open_status', 'ticket_no', 'customer_name', 'ticket_tat', 'certificate_no', 'textbody', 'subject', 'messagedate','load_date').drop_duplicates()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
df_temp=df_temp.withColumn('concern',concat_ws(' ',col('classification_disposition__c'),col('sub_deposition')))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
df_temp=df_temp.select('golden_id','concern','customer_code', 'mobile','ticket_status', 'ticket_stage', 'assigned_team', 'resolution_remarks', 'resolution_date', 'ticket_source', 'interaction_date', 'ticket_close_date', 'status_closed_date__c', 'email', 'policy_no', 'remarks', 'open_status', 'ticket_no', 'customer_name', 'ticket_tat', 'certificate_no', 'textbody', 'subject', 'messagedate','load_date').drop_duplicates()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
df_temp.persist()
records=df_temp.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
output_index = "summary_mart"
schema = "customermart"


import sys
try:
    
    df_temp.write.format("greenplum")\
    .option("dbtable",output_index).option('dbschema',schema)\
    .option("server.port","1150-1170").option("url",prod_url)\
    .option("user", user_prod).option("password",pwd_prod).mode('append').save()
    
    
except Exception as e:
    x = e
    print(x)
else:
    x = 200 #success
    print(x)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

200

In [54]:
table_name='gen_ai_summary'
source='gen_ai_summary'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
max_commit_timestamp =df_temp.select(max('load_date')).first()[0]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
max_commit_timestamp

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

datetime.datetime(2024, 3, 22, 0, 0)

In [57]:
if  x == 200:
    update_progress(table_name,source,time_filter,max_commit_timestamp,records,start_time,starttime,'Success')
    print('SUCCESS')
else:
    update_progress(table_name,source,time_filter,max_commit_timestamp,'0',start_time,starttime,'Failed')
    print('FAILED')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Updated Progress for gen_ai_summary from gen_ai_summary until 2024-03-22 00:00:00. 5278508 records processed in this run
success
SUCCESS

In [63]:
from datetime import datetime,timedelta
filter_date=datetime.now()-timedelta(days=31)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [64]:
filter_date

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

datetime.datetime(2024, 2, 21, 23, 33, 51, 45499)

In [69]:
gscPythonOptions = {
         "url": "jdbc:postgresql://10.35.12.194:5432/gpadmin","user": "gpcustcoe","password": "gpcustcoe@123",
         "dbschema": "customermart","dbtable": "summary_mart",
         "server.port":"1150-1170"} 
df_summary =sqlContext.read.format("greenplum").options(**gscPythonOptions).load().filter(col('interaction_date')>=filter_date)\
.drop_duplicates()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [70]:
from pyspark.sql.window import Window

df_summary = df_summary.withColumn("rn", F.row_number()
        .over(Window.partitionBy('golden_id','ticket_no')
        .orderBy(F.col("interaction_date").desc(),F.col("messagedate").desc())))

df_summary = df_summary.filter(F.col("rn")<=10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [71]:
df_summary.persist()
df_summary.count()
df_temp.unpersist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[golden_id: int, concern: string, customer_code: string, mobile: string, ticket_status: string, ticket_stage: string, assigned_team: string, resolution_remarks: string, resolution_date: string, ticket_source: string, interaction_date: string, ticket_close_date: string, status_closed_date__c: string, email: string, policy_no: string, remarks: string, open_status: string, ticket_no: string, customer_name: string, ticket_tat: string, certificate_no: string, textbody: string, subject: string, messagedate: string, load_date: timestamp]

In [72]:
df_summary.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['golden_id', 'concern', 'customer_code', 'mobile', 'ticket_status', 'ticket_stage', 'assigned_team', 'resolution_remarks', 'resolution_date', 'ticket_source', 'interaction_date', 'ticket_close_date', 'status_closed_date__c', 'email', 'policy_no', 'remarks', 'open_status', 'ticket_no', 'customer_name', 'ticket_tat', 'certificate_no', 'textbody', 'subject', 'messagedate', 'load_date', 'rn']

In [73]:
df_transformed=df_summary.rdd.map(lambda row:(row.golden_id,{"concern":row.concern,\
"customer_code":row.customer_code,"mobile":row.mobile,"ticket_status":row.ticket_status,"ticket_stage":row.ticket_stage,\
"assigned_team":row.assigned_team,"resolution_remarks":row.resolution_remarks,\
"resolution_date":row.resolution_date,"ticket_source":row.ticket_source,"interaction_date":row.interaction_date,\
"ticket_close_date":row.ticket_close_date,"status_closed_date__c":row.status_closed_date__c,"email":row.email,\
"policy_no":row.policy_no,"remarks":row.remarks,"open_status":row.open_status,"ticket_no":row.ticket_no,\
"customer_name":row.customer_name,"ticket_tat":row.ticket_tat,"certificate_no":row.certificate_no,"textbody":row.textbody,\
"subject":row.subject,"messagedate":row.messagedate}))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [74]:
grouped_data=df_transformed.groupByKey().mapValues(list)
df_final=grouped_data.toDF(["golden_id","data"])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [75]:
df_final.persist()
df_summary.unpersist()
records=df_final.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [76]:
records

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

178569

In [77]:
target_index='get_sos_details'
print(target_index)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

get_sos_details

In [78]:
import json, pprint, requests
es_nodes = '10.35.12.212'
es_port = '9200'
es_user = 'elastic'
es_pwd = 'bEiilauM3es'

######target_index = 'actuarial_premium_risk_tables'----previous name...delete later
# get_sos_details
#alias_name = 'nepuepr_uat_3'
target_doc_type = 'get_sos_details'
#########delete index if already exists########################################################################################################################################################################

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# #########delete index ###############################################
# headers = {'Content-Type': 'application/json'}
# query = {"query":{"match_all":{}}}
# URL = 'http://'+es_user+':'+es_pwd+'@'+es_nodes+':'+es_port+'/'+target_index
# r = requests.delete(URL, data=json.dumps(query), headers=headers) 
# print(r)

In [ ]:
# #########create index ##################################################################################################################################################################
# headers = {'Content-Type': 'application/json'}
# query_create= {"settings": { "index.number_of_shards": "1" }}
# URL = 'http://'+es_user+':'+es_pwd+'@'+es_nodes+':'+es_port+'/'+target_index
# r = requests.put(URL, data=json.dumps(query_create), headers=headers)
# print(r)

In [79]:
target_index_doc_type = target_index +'/'+ target_doc_type
target_index_doc_type

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'get_sos_details/get_sos_details'

In [80]:
try:
     df_final.write.format('org.elasticsearch.spark.sql').option('es.index.auto.create', 'true')\
        .option('es.nodes' , es_nodes).option('es.port', es_port).option('es.resource', target_index_doc_type)\
        .mode("overwrite").save() 
except Exception as e :
    x = e
else: 
    x = 200 # success
print(x)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

200

In [81]:
table_name='summary_mart'
source='summary_mart'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [82]:
if  x == 200:
    update_progress(table_name,source,time_filter,max_commit_timestamp,records,start_time,starttime,'Success')
    print('SUCCESS')
else:
    update_progress(table_name,source,time_filter,max_commit_timestamp,'0',start_time,starttime,'Failed')
    print('FAILED')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Updated Progress for summary_mart from summary_mart until 2024-03-22 00:00:00. 178569 records processed in this run
success
SUCCESS

In [83]:
spark.stop()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…